In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/StrawberryLeaf.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
!pip install -q keras

In [ ]:
import os
import random
import zipfile
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
#Import All imperative function for "Tensorflow", CNN
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
directory = '/content/StrawberryLeaf'

#now we want to extract the cat and dog folder

data = []
categories = ['Strawberry___healthy', 'Strawberry___Leaf_scorch']

for i in categories:
    
    folder_path = os.path.join(directory,i)
    
    for image in os.listdir(folder_path):
        label = categories.index(i)
        filepath_ = os.path.join(folder_path, image)
        img_arr_  = cv2.imread(filepath_)
        img_arr_  = cv2.resize(img_arr_, (100,100))
        data.append([img_arr_, label])


random.shuffle(data)

In [ ]:
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

In [ ]:
X = np.array(X)
y = np.array(y)

X = X/255

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation = 'relu', input_shape = (X.shape[1:])))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X, y, epochs = 20,batch_size = 32, validation_split = 0.20)

Hyper Parameter Tuning

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):

  model = Sequential()

  #Create first convulation layer with MaxPool2D
  model.add(Conv2D(64, (3,3), activation = 'relu', input_shape = (X.shape[1:])))
  model.add(MaxPool2D(2,2))
  model.add(BatchNormalization())

  #Create second Convolation layer with MaxPool2d
  model.add(Conv2D(64, (3,3), activation = 'relu'))
  model.add(MaxPool2D(2,2))
  model.add(BatchNormalization())
  
  #Create kernel_initializer
  kernel_initializer_ = ['uniform', 'he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal']
  
  #Create Third covlulation layer with maxPool2D
  model.add(Conv2D(64, (3,3), activation = 'relu'))
  model.add(MaxPool2D(2,2))
  model.add(BatchNormalization())
  
  #Flatten the whole convulation
  model.add(Flatten())
  
  #Fully connected layer or ANN
  model.add(Dense(128, activation = 'relu', kernel_initializer = hp.Choice('kernel1_', values = kernel_initializer_)))
  model.add(Dropout(hp.Choice('dropoutLayer1_',values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
  model.add(BatchNormalization())

  #Fully Connected Layer or ANN
  model.add(Dense(128, activation = 'relu', kernel_initializer = hp.Choice('Kernel2_', values = kernel_initializer_)))
  model.add(Dropout(hp.Choice('dropoutLayer2_',values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
  model.add(BatchNormalization())

  #Output Layer
  model.add(Dense(1, activation = 'sigmoid'))

  #Compile
  optimizer_ = hp.Choice('optimizer_', values = ['RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
  model.compile(optimizer = optimizer_, loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [ ]:
tunner_ = kt.RandomSearch(hypermodel   = build_model, 
                          objective    = 'val_accuracy',
                          max_trials   = 5,
                          directory   = 'lealf3_'
                         )

In [ ]:
tunner_.search(X, y, epochs = 5, validation_split = 0.2)

In [ ]:
tunner_.get_best_hyperparameters()[0].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
tuneModel = tunner_.get_best_models(num_models=1)[0]
tuneModel.fit(X, y, batch_size = 32, epochs= 55, initial_epoch = 6, validation_split = 0.2, validation_data=(x_test, y_test))

In [ ]:
print('accuracy is = ', tuneModel.evaluate(x_test, y_test)[1]*100,'%')